## Importações

In [1]:
import numpy as np
from random import randint

## Funções de Matrizes

In [2]:
def Add(matrix_a,matrix_b):
    return np.add(matrix_a,matrix_b)

def Subtract(matrix_a,matrix_b):
    return np.subtract(matrix_a,matrix_b)
    
def Multiply(matrix_a, matrix_b):
    matrix_a = np.asmatrix(matrix_a)  
    matrix_b = np.asmatrix(matrix_b)
    matrix = matrix_a * matrix_b
    return np.asmatrix(matrix)

def Sigmoid(x):
    return 1/(1 + np.exp(-(x)))

def D_sigmoid(x):
    # pegadno as dimensões
    matrix_a = np.asmatrix(x)  
    linhas = matrix_a.shape[0]
    colunas = matrix_a.shape[1]
    
    #matrix resultante
    matrix_aux = []
    #percorrendo as linhas e colunas da matrix de entrada
    for l in range(linhas):
        linhas_aux = []
        for c in range(colunas):
            #aplicando a derivada para cada valor da matrix de entrada
            sigmoid = matrix_a[l,c] * (1 - matrix_a[l,c] )
            linhas_aux.append(sigmoid)
            
        matrix_aux.append(linhas_aux)
    
    return np.asmatrix(matrix_aux)

def Escalar_multiply(a,b):
    matrix = np.asmatrix(a)
    matrix = matrix * b
    return matrix

def Hadamard(matrix_a,matrix_b):
    matrix_a = np.asmatrix(matrix_a)
    matrix_b = np.asmatrix(matrix_b)
    matrix = np.multiply(matrix_a,matrix_b)
    return matrix

# Criando a classe Rede Neural

In [3]:
class RedeNeural():
    def __init__(self,Nodes,Camadas,Saida):
        self.i_nodes = Nodes
        self.h_nodes = Camadas
        self.o_nodes = Saida
        
        self.bias_ih = np.random.random((1,Camadas))
        self.bias_ho = np.random.random((Saida,1))        

        self.weigth_ih = np.random.random((self.i_nodes, self.h_nodes))
        self.weigth_ho = np.random.random((self.o_nodes, self.h_nodes))
        
        self.learning_rate = 0.1

    
    def train(self,entrada,target):
        # entrada
        entrada = np.asmatrix(entrada)
        hidden = Multiply(entrada,self.weigth_ih)
        hidden = Add(hidden,self.bias_ih)
        hidden = Sigmoid(hidden)
        
        #saida
        saida = Multiply(self.weigth_ho,hidden.T)
        saida = Add(saida,self.bias_ho)
        saida = Sigmoid(saida)
        
        #
        #print(saida)
        
        #BackPropagation
        # saida para oculta corrigidos
        expected = np.asmatrix(target)
        output_error = Subtract(expected,saida.T)
        d_output = D_sigmoid(saida)       
        
        gradiente = Hadamard(output_error,d_output.T)
        gradiente = Escalar_multiply(gradiente,self.learning_rate)
        
        #ajuste dos Bias_HO
        self.bias_ho = Add(self.bias_ho, gradiente.T)
        
        # ajuste dos pesos
        weigth_ho_deltas = Multiply(gradiente.T,hidden)
        self.weigth_ho = Add(self.weigth_ho, weigth_ho_deltas)
      
        # Oculta para saida
        hidden_error = Multiply(output_error,self.weigth_ho)
        d_hidden = D_sigmoid(hidden)
        
        gradiente_H = Hadamard(hidden_error,d_hidden)
        gradiente_H = Escalar_multiply(gradiente_H,self.learning_rate)
        
        #ajuste dos Bias_IH
        self.bias_ih = Add(self.bias_ih, gradiente_H)
        
        # ajuste dos pesos
        weigth_ih_deltas = Multiply(gradiente_H.T,entrada)
        self.weigth_ih = Add(self.weigth_ih, weigth_ih_deltas.T)
        
    # função de previsão de dados
    def predict(self,array):
        # entrada
        entrada = np.asmatrix(array)
        hidden = Multiply(entrada,self.weigth_ih)
        hidden = Add(hidden,self.bias_ih)
        hidden = Sigmoid(hidden)
        
        #saida
        saida = Multiply(self.weigth_ho,hidden.T)
        saida = Add(saida,self.bias_ho)
        saida = Sigmoid(saida)
        saida = saida.tolist()
        return saida[0][0]

# Testando em casos reais

In [4]:
import pandas as pd

In [5]:
dados = pd.read_csv("Admission_Predict.csv")
dados.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


### Normalização

In [6]:
dados['GRE Score'] = (dados['GRE Score'] - dados['GRE Score'].min()) / (dados['GRE Score'].max() - dados['GRE Score'].min())
dados['TOEFL Score'] = (dados['TOEFL Score'] - dados['TOEFL Score'].min()) / (dados['TOEFL Score'].max() - dados['TOEFL Score'].min())
dados['University Rating'] = (dados['University Rating'] - dados['University Rating'].min()) / (dados['University Rating'].max() - dados['University Rating'].min())
dados['SOP'] = (dados['SOP'] - dados['SOP'].min()) / (dados['SOP'].max() - dados['SOP'].min())
dados['LOR '] = (dados['LOR '] - dados['LOR '].min()) / (dados['LOR '].max() - dados['LOR '].min())
dados['CGPA'] = (dados['CGPA'] - dados['CGPA'].min()) / (dados['CGPA'].max() - dados['CGPA'].min())
dados['Research'] = (dados['Research'] - dados['Research'].min()) / (dados['Research'].max() - dados['Research'].min())
dados['Chance of Admit '] = (dados['Chance of Admit '] - dados['Chance of Admit '].min()) / (dados['Chance of Admit '].max() - dados['Chance of Admit '].min())

In [7]:
dados.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,0.94,0.928571,0.75,0.875,0.875,0.913462,1.0,0.920635
1,2,0.68,0.535714,0.75,0.750,0.875,0.663462,1.0,0.666667
2,3,0.52,0.428571,0.50,0.500,0.625,0.384615,1.0,0.603175
3,4,0.64,0.642857,0.50,0.625,0.375,0.599359,1.0,0.730159
4,5,0.48,0.392857,0.25,0.250,0.500,0.451923,0.0,0.492063


## Definindo classes de Aprovação (1) ou Reprovação (0)

In [8]:
media = dados['Chance of Admit '].mean()
dados[(dados[["Chance of Admit "]] >= media)] = 1
dados[(dados[["Chance of Admit "]] <= media)] = 0
dados.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,0.94,0.928571,0.75,0.875,0.875,0.913462,1.0,1.0
1,2,0.68,0.535714,0.75,0.750,0.875,0.663462,1.0,1.0
2,3,0.52,0.428571,0.50,0.500,0.625,0.384615,1.0,0.0
3,4,0.64,0.642857,0.50,0.625,0.375,0.599359,1.0,1.0
4,5,0.48,0.392857,0.25,0.250,0.500,0.451923,0.0,0.0


## Selecionando as colunas para as entradas

In [9]:
# Armazenando resultado esperavel na variavel Y
Y = dados['Chance of Admit '].values

In [10]:
# Armazenando resultado esperavel na variavel X
X = dados.iloc[:, 1:6].values

In [11]:
# dividindo em grupos de Teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

### Inicializando a Rede

In [12]:
rede = RedeNeural(5,5,1)

### Treinando a Rede

In [13]:
#numero de epocas
epocas = 100

for ep in range (epocas):
    for index in range(len(X_train)):
        rede.train(X_train[index], y_train[index])

## Verificando a taxa de acerto

In [14]:
erro = 0
acerto = 0
for c in range(len(X_test)):
    if (rede.predict(X_test[c]) > 0.5) :
        if (y_test[c] == 1.0):
            acerto += 1
        else:
            erro +=1
    else:
        if (y_test[c] == 0.0):
            acerto += 1
        else:
            erro +=1
        
print("Acerto: ", acerto, " ou ", "%.2f"%((acerto*100)/len(X_test)),"%")
print("Erro: ", erro, " ou ", "%.2f"%((erro*100)/len(X_test)),"%")


Acerto:  103  ou  85.83 %
Erro:  17  ou  14.17 %
